In [1]:
from MT5 import *
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import time
import pickle
import ta
from joblib import dump, load
import os
from sklearn.preprocessing import StandardScaler
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


path = "" # Ex: C:/Desktop/Python_for_finance_and_algorithmic_trading/ChapterN/Models



def feature_engineering(df):
    """ DON'T PUT THE SHIFT HERE"""
    
    # We copy the dataframe to avoid interferences in the data
    df_copy = df.copy()
    df_copy[["returns"]] = df_copy[["close"]].pct_change(1)

    # Create the SMAs
    df_copy["SMA 15"] = df_copy[["close"]].rolling(15).mean()
    df_copy["SMA 60"] = df_copy[["close"]].rolling(60).mean()
    
    # Create the volatilities
    df_copy["MSD 10"] = df_copy[["returns"]].rolling(10).std()
    df_copy["MSD 30"] = df_copy[["returns"]].rolling(30).std()
    
    # Create the Rsi
    RSI = ta.momentum.RSIIndicator(df_copy["close"], window= 14, fillna = False)
    df_copy["rsi"] = RSI.rsi()
    
    # STANDARDIZATION

    sc = StandardScaler()
    df_copy = df_copy[["SMA 15", "SMA 60", "MSD 10", "MSD 30", "rsi"]].dropna()
    df_copy_sc = sc.fit_transform(df_copy)

    return df_copy_sc


def ANN_weights():
    nb_hidden_layer=2
    # INTIALIZATION SEQUENTIAL MODEL
    regressor = Sequential()

    # FIRST LAYER
    regressor.add(Dense(25, input_shape=(5,), activation="relu"))

    # ADD HIDDEN LAYER
    for i in range(1,nb_hidden_layer):
        regressor.add(Dense(int(25/i), activation="relu"))

    # OUTPUT LAYER DENSE
    regressor.add(Dense(1, activation="linear"))

    # COMPILE THE MODEL
    regressor.compile(loss="mse", optimizer="adam")

    return regressor

   
    
def ANN_reg_sig(symbol):
    """ Function for predict the value of tommorow using DNN model"""
    
    # Create the weights if there is not in the folder
    ann_1 = ANN_weights()
    ann_2 = ANN_weights()
    ann_3 = ANN_weights()

    # Import trained weights
    ann_1.load_weights(f"ANN_weights/ANN n17")
    ann_2.load_weights(f"ANN_weights/ANN n38")
    ann_3.load_weights(f"ANN_weights/ANN n66")
    
    # Take the lastest percentage of change 
    df = MT5.get_data(symbol, 70)
    
    # Features engeeniring
    data = feature_engineering(df)

    X = data[-1:,:]
    
    # Bagging
    pr1 = np.sign(ann_1.predict(X))
    pr2 = np.sign(ann_2.predict(X))
    pr3 = np.sign(ann_3.predict(X))
    
    # Find the signal
    buy = (pr1 + pr2 + pr3)[0][0] >=1
    sell = not buy
    
    
    return buy, sell



# True = Live Trading and False = Screener
live = True

if live:
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")
    print("------------------------------------------------------------------")



info_order = {
    "CADGBP": ["GBPCAD", 1.00]
}


start = datetime.now().strftime("%H:%M:%S")
while True:
    # Verfication for launch
    if datetime.now().weekday() not in (5,1):
        is_time = datetime.now().strftime("%H:%M:%S") == start #"23:59:59"
    else:
        is_time = False

    
    # Launch the algorithm
    if is_time:

        # Open the trades
        for asset in info_order.keys():

            # Initialize the inputs
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Create the signals
            buy, sell = ANN_reg_sig(symbol)

             # Run the algorithm
            if live:
                MT5.run(symbol, buy, sell,lot)

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {buy}\t"
                     f"Sell: {sell}")
    time.sleep(1)

------------------------------------------------------------------
Date:  2022-02-17 08:20:54
Balance: 8990.52 USD, 	Equity: 8990.52 USD, 	Profit: 0.0 USD
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2022-02-17 08:20:55
SYMBOL: GBPCAD
BUY: False 	  SHORT: True
POSITION: None 	 ID: None
OPEN SHORT TRADE: Request executed
------------------------------------------------------------------


KeyboardInterrupt: 